## Generate recommendation statistics

### With MoviesLens dataset

### With Jester Joke dataset

In [ ]:
import main_jester_joke

In [ ]:
main_jester_joke.main()

### With GoodReads Dataset

In [1]:
import main_goodreads

In [ ]:
main_goodreads.main()